In [ ]:
import os
import numpy as np
import time

import torch
from torch_geometric.data import Dataset, Data

import import_ipynb
from simulation import simul_fromData, simul
from constants import *
from utils import set_gpu, load_model, adj2Data

In [ ]:
def greedy_orig(adj_list_tup, seed_idx, del_edge_num, simul_num=SIMUL_NUM):
    time_start = time.time()
    time_alg = []
    n = len(adj_list_tup)
    
    # since dataset's adj list is of type tuple, convert it to list
    adj_list = [[[v,p] for v,p in adj_list_tup[u]] for u in range(n)]
    
    mask = []
    for _ in range(del_edge_num):
        pred_min = float('inf')
        idx_min = None
        for u in range(n):
            for i,(v,p) in enumerate(adj_list[u]):
                if p==0: continue
                adj_list[u][i][1]=0
                pred = sum(simul(adj_list,seed_idx,simul_num))
                adj_list[u][i][1]=p
                if pred<pred_min: pred_min = pred; idx_min=(u,v,i,p)
        mask.append([idx_min[0],idx_min[1],idx_min[3]])
        adj_list[idx_min[0]][idx_min[2]][1]=0

        time_alg.append(time.time()-time_start)
    return mask, time_alg